# Library import

In [ ]:
import os
import subprocess
import time

import yaml
from sklearn.model_selection import GroupKFold

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch.nn as nn

from pathlib import Path
import json
from collections import defaultdict, Counter
import random
import random
import shutil
from tqdm import tqdm
import zipfile
from collections import Counter

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

# Path

In [ ]:
COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_COCO_JSON_NM = 'mod_COCO_annotations.json'
OUT_IMAGE_FLDR_NM = 'images'
TRAIN_NM = 'train'
VAL_NM = 'val'
TEST_NM = 'test'
LABEL_NM = 'labels'
YAML = 'dataset.yaml'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
yolo_dataset_pth = Path('/kaggle/input/yolodataset-xview')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')

coco_json_pth = in_dataset_pth / COCO_JSON_NM
new_coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM

labels_pth = yolo_dataset_pth / LABEL_NM
train_img = yolo_dataset_pth / TRAIN_NM
val_img = yolo_dataset_pth / VAL_NM
test_img = yolo_dataset_pth / TEST_NM
train_txt = labels_pth / TRAIN_NM
val_txt = labels_pth / VAL_NM
test_txt = labels_pth / TEST_NM
dataset_yaml = yolo_dataset_pth / YAML

In [ ]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

# Utility

In [ ]:
def load_json(file_path):
    """
    Carica un file JSON dal percorso specificato.

    :param file_path: Percorso al file JSON da caricare.
    :return: Dati contenuti nel file JSON (come dizionario o lista).
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# COCO Preprocessing

In [ ]:
def process_custom_coco_json(input_path, output_path):
    """
    Funzione per processare un JSON COCO in formato personalizzato.
    """
    # Leggi il JSON dal file di input
    data = load_json(input_path)

    # Ottieni e correggi il formato delle categorie
    raw_categories = data.get('categories', [])
    categories = []
 
    for category in tqdm(raw_categories, desc="Processing Categories"):
        for id_str, name in category.items():
            try:
                categories.append({"id": int(id_str), "name": name})
            except ValueError:
                print(f"Errore nel parsing della categoria: {category}")
 
    # Preprocessa le annotazioni in un dizionario per immagini
    image_annotations_dict = {}
    for annotation in tqdm(data.get('annotations', []), desc="Building Image Annotations Dictionary"):
        image_id = annotation['image_id']
        if image_id not in image_annotations_dict:
            image_annotations_dict[image_id] = []
        image_annotations_dict[image_id].append(annotation)
 
    # Elenco di annotazioni da mantenere (solo quelle valide)
    valid_annotations = []
    annotations_to_remove = set()
 
    # Controllo dei bounding box
    for annotation in tqdm(data.get('annotations', []), desc="Processing Annotations"):
        
        # Converte il formato del bbox
        if isinstance(annotation['bbox'], str):
            annotation['bbox'] = json.loads(annotation['bbox'])
        
        x, y, width, height = annotation['bbox']
        xmin, xmax = x, x + width
        ymin, ymax = y, y + height
        
        # Verifica che xmin < xmax e ymin < ymax, e che la larghezza e altezza siano sufficienti
        if xmin >= xmax or ymin >= ymax or width <= 10 or height <= 10:
            annotations_to_remove.add(annotation['id'])
        else:
            annotation['bbox'] = [xmin, ymin, xmax, ymax]
            valid_annotations.append(annotation)
 
    # Rimuovi le annotazioni non valide
    data['annotations'] = valid_annotations
 
    # Ordina le categorie per ID
    categories = sorted(categories, key=lambda x: x['id'])
    
    # Aggiorna le categorie nel JSON
    data['categories'] = categories

 
    # Scrivi il JSON modificato nel file di output
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
process_custom_coco_json(coco_json_pth, new_coco_json_pth)

### Category Check

In [ ]:
def count_bounding_boxes(json_path):
    """
    Conta il numero di bounding box per ogni categoria in un file COCO JSON.

    Args:
        json_path (str): Percorso del file JSON.

    Returns:
        list: Elenco di tuple con ID categoria, nome categoria e numero di bounding box.
    """
    # Carica il file JSON
    coco_data = load_json(json_path)

    # Estrarre i dati principali
    annotations = coco_data.get("annotations", [])
    categories = coco_data.get("categories", [])

    # Mappare id di categoria ai nomi delle categorie
    category_id_to_name = {category["id"]: category["name"] for category in categories}

    # Contare i bounding box per categoria
    bbox_counts = defaultdict(int)
    for annotation in annotations:
        category_id = annotation["category_id"]
        bbox_counts[category_id] += 1

    # Creare un elenco dei risultati
    results = [
        (cat_id, category_id_to_name.get(cat_id, "Unknown"), count)
        for cat_id, count in bbox_counts.items()
    ]
    
    # Ordinare i risultati in ordine decrescente per numero di bounding box
    results.sort(key=lambda x: x[2], reverse=True)
    
    # Stampare i risultati
    for cat_id, category_name, count in results:
        print(f"Categoria ID {cat_id} ('{category_name}'): {count} bounding box")

In [ ]:
count_bounding_boxes(new_coco_json_pth)

# JSON to YOLO

In [ ]:
def convert_json_to_yolo(json_path, images_dir, output_dir, input_dir, train_dir_out, val_dir_out, test_dir_out, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il file JSON
    with open(json_path) as f:
        data = json.load(f)

    # Mappa le classi
    class_mapping = {category['id']: category['name'] for category in data['categories']}
    nc = len(class_mapping)  # Numero di classi

    # Crea le cartelle per il dataset
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')
    labels_dir = os.path.join(output_dir, 'labels')
    
    for dir_path in [train_dir, val_dir, test_dir, labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Dividi le immagini in training, validation, and test
    images = data['images']
    random.shuffle(images)
    total_images = len(images)
    
    train_split = int(train_ratio * total_images)
    val_split = int((train_ratio + val_ratio) * total_images)

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    # Funzione per copiare le immagini in una cartella specifica
    def copy_images(image_list, target_dir):
        for image in tqdm(image_list, desc=f"Copying images to {target_dir}", unit="image"):
            src_path = os.path.join(images_dir, image['file_name'])
            dst_path = os.path.join(target_dir, image['file_name'])
            shutil.copy(src_path, dst_path)

    # Copia le immagini nelle rispettive cartelle
    copy_images(train_images, train_dir)
    copy_images(val_images, val_dir)
    copy_images(test_images, test_dir)

    # Converte le annotazioni in formato YOLO e salva nei file di testo
    def convert_annotations(image, annotations, target_dir):
        image_id = image['id']
        image_width = image['width']
        image_height = image['height']
        image_name = image['file_name']

        label_file_path = os.path.join(target_dir, f"{image_name.replace('.jpg', '.txt')}")
        label_dir = os.path.dirname(label_file_path)

        # Crea la cartella se non esiste
        os.makedirs(label_dir, exist_ok=True)

        with open(label_file_path, 'w') as label_file:
            for annotation in annotations:
                if annotation['image_id'] == image_id:
                    category_id = annotation['category_id']
                    xmin, ymin, xmax, ymax = annotation['bbox']

                    # Normalizza le coordinate
                    x_center = (xmin + xmax) / 2 / image_width
                    y_center = (ymin + ymax) / 2 / image_height
                    width = (xmax - xmin) / image_width
                    height = (ymax - ymin) / image_height

                    # Scrivi nel file di annotazione
                    label_file.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

    # Converte le annotazioni per ogni immagine e le salva nelle cartelle appropriate
    def process_images(image_list, target_dir, label_target_dir):
        for image in tqdm(image_list, desc=f"Converting annotations", unit="image"):
            annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] == image['id']]
            convert_annotations(image, annotations, os.path.join(label_target_dir, target_dir))

    # Processa le immagini per training, validation e test
    process_images(train_images, 'train', labels_dir)
    process_images(val_images, 'val', labels_dir)
    process_images(test_images, 'test', labels_dir)
        
    # Crea il file YAML per YOLO
    yaml_content = f"""path: {input_dir}
train: {train_dir_out}  # train images
val: {val_dir_out}  # val images
test: {test_dir_out}  # test images

# Classes
nc: {nc}  # number of classes
names:
"""
    # Aggiungi le classi al file YAML
    for idx, class_name in class_mapping.items():
        yaml_content += f"  {int(idx)}: {class_name}\n"  # Assicurati che l'indice sia numerico senza virgolette

    # Scrivi il file YAML
    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as yaml_file:
        yaml_file.write(yaml_content.strip())

    print("Conversione e split completati.")

In [ ]:
convert_json_to_yolo(
    json_path=new_coco_json_pth,
    images_dir=img_fldr,
    output_dir=out_dataset_pth,
    input_dir=yolo_dataset_pth,
    train_dir_out=train_img,
    val_dir_out=val_img,
    test_dir_out=test_img,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1
)

In [ ]:
# Nome del file zip da creare
zip_file_name = "YOLO_dataset.zip"

# Elenco di file e cartelle da includere nello zip
items_to_zip = [
    "labels",
    "test",
    "train",
    "val",
    "dataset.yaml",
]

# Funzione per aggiungere file e cartelle allo zip
def zip_folder(zipf, folder_path, base_folder=""):
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, base_folder)
            zipf.write(file_path, arcname)

# Creazione dello zip
with zipfile.ZipFile(zip_file_name, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    for item in items_to_zip:
        if os.path.exists(item):  # Verifica che il file o la cartella esista
            if os.path.isdir(item):  # Se è una cartella, aggiungi tutto il contenuto
                zip_folder(zipf, item, out_dataset_pth)
            else:  # Se è un file, aggiungilo direttamente
                zipf.write(item)
        else:
            print(f"Elemento non trovato: {item}")

## Check Yaml

In [ ]:
def analyze_yolo_dataset(yaml_path):
    # Carica il file YAML
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # Estraggo le informazioni dal file YAML
    base_path = data['path']
    sets = ['train', 'val', 'test']
    stats = {}
    
    total_images = 0
    total_bboxes = Counter()
    class_names = data['names']

    for dataset_set in sets:
        images_path = os.path.join(base_path, dataset_set)
        labels_path = os.path.join(base_path, 'labels', dataset_set)

        # Controlla se le directory esistono
        if not os.path.exists(images_path) or not os.path.exists(labels_path):
            print(f"Directory per il set '{dataset_set}' non trovata.")
            continue

        # Conta immagini
        image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        num_images = len(image_files)
        total_images += num_images
        
        stats[dataset_set] = {"images": num_images, "bbox": Counter()}

        # Conta bounding box con tqdm
        print(f"Analizzando {dataset_set}...")
        for label_file in tqdm(os.listdir(labels_path), desc=f"Contando bbox in {dataset_set}", unit="file"):
            label_path = os.path.join(labels_path, label_file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = line.split()[0]  # La classe è il primo elemento della riga
                    stats[dataset_set]["bbox"][class_id] += 1
                    total_bboxes[class_id] += 1

    return stats, total_images, total_bboxes, class_names

# Funzione per mostrare i risultati con le percentuali calcolate per set
def print_stats(stats, total_images, total_bboxes, class_names):
    print("\n--- ANALISI COMPLESSIVA ---")
    print(f"Totale immagini: {total_images}")
    print(f"Distribuzione complessiva delle categorie:")
    
    # Ordina le classi in base al numero di bbox, dal più grande al più piccolo
    sorted_bboxes = sorted(total_bboxes.items(), key=lambda x: x[1], reverse=True)
    
    for class_id, count in sorted_bboxes:
        percentage = (count / sum(total_bboxes.values())) * 100
        print(f"  Classe {class_names[int(class_id)]} ({class_id}): {count} bbox ({percentage:.2f}%)")
    
    print("\n--- ANALISI PER SET ---")
    
    total_bboxes_all_sets = 0  # Per calcolare il totale complessivo dei bbox
    for dataset_set, data in stats.items():
        images_percentage = (data['images'] / total_images) * 100 if total_images > 0 else 0
        print(f"\n--- {dataset_set.upper()} ---")
        print(f"Numero di immagini: {data['images']} ({images_percentage:.2f}%)")
        print("Distribuzione categorie:")
        
        # Calcola il totale dei bbox per ogni set
        total_bboxes_in_set = sum(data['bbox'].values())
        total_bboxes_all_sets += total_bboxes_in_set  # Aggiungi al totale complessivo
        
        # Ordina le classi per ogni set in base al numero di bbox
        sorted_set_bboxes = sorted(data['bbox'].items(), key=lambda x: x[1], reverse=True)
        
        for class_id, count in sorted_set_bboxes:
            percentage = (count / total_bboxes_in_set) * 100 if total_bboxes_in_set > 0 else 0
            print(f"  Classe {class_names[int(class_id)]} ({class_id}): {count} bbox ({percentage:.2f}%)")
        
        print("Totale bounding box:", total_bboxes_in_set, f"({(total_bboxes_in_set / sum(total_bboxes.values())) * 100:.2f}%)")

    # Aggiungi la percentuale per il totale complessivo dei bounding box
    print("\n--- TOTALE COMPLESSIVO ---")
    print(f"Totale bounding box: {total_bboxes_all_sets}")

In [ ]:
# Esegui analisi
stats, total_images, total_bboxes, class_names = analyze_yolo_dataset(dataset_yaml)
print_stats(stats, total_images, total_bboxes, class_names)

# YoloV11

In [ ]:
# Pretrained Model
model = YOLO('yolo11n.pt') 

# Configurazione del training
results = model.train(
    data=dataset_yaml,     # Path al file YAML del dataset
    epochs=3,              # Numero di epoche
    batch=64,              # Dimensione del batch
    imgsz=320,             # Dimensione delle immagini (320x320)
    lr0=0.001,             # Learning rate iniziale
    save=True,             # Salva i pesi migliori
)

In [ ]:
results = model.val()  # evaluate model performance on the validation set